##Обязательная часть
Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>
В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

Дополнительная часть (необязательная)
Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

list_of_search_queries = ['python', 'анализ данных']
number_of_pages = 2


def pars(list_of_search_queries=['анализ данных', 'python'], number_of_pages=1):
  url = 'https://habr.com/ru/search/'
  page = 'page'
  habr = 'https://habr.com'

  res_df = pd.DataFrame(columns=['date', 'title', 'href', 'likes', 'text'])


  for name in list_of_search_queries:
    

    for number in range(1, number_of_pages + 1):
      
      link = url + page + str(number) + '/'
      res = requests.get(link, {f'q': {name},
                                'target_type': 'posts',
                                'order': 'date'})
      soup = BeautifulSoup(res.text)
      articles = soup.find_all('article', class_="tm-articles-list__item")

      for article in articles:
        
        title = article.find('a', class_="tm-article-snippet__title-link")
        href = habr + title.attrs.get('href')
        dat = article.find('time').attrs.get('title')
        likes = article.find('div', class_="tm-votes-meter tm-data-icons__item").text.split(' ')
        likes = likes[-1]

        req = requests.get(href)
        soup = BeautifulSoup(req.text)
        text = soup.find('div', id="post-content-body").text

        row = {'date': dat, 'title': title.text, 'href': href, 'likes': likes, 'text': text}

        res_df = pd.concat([res_df, pd.DataFrame([row])]) 

  res_df = res_df.drop_duplicates()
  return res_df.reset_index(drop=True)


  
pars(list_of_search_queries, number_of_pages)
